In [1]:
%pip install mwparserfromhell
%pip install lxml
%pip install pyspark
%pip install nltk
%pip install numba

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Various imports

In [23]:
import csv
import nltk
import re
import os.path
import shutil
import mwparserfromhell
from lxml import etree
from nltk.corpus import stopwords
from tqdm import tqdm
from alive_progress import alive_bar
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, lower, split, count

Static things

In [24]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
header = ['Index', 'Text']
lista = []

#Testando una la alive progress bar
num_rows = 356900000
counter = 0
increment = 100000

[nltk_data] Downloading package stopwords to /home/fabbro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dynamic things

In [5]:
f= open("file.csv", "w+")
writer = csv.writer(f)
writer.writerow(header)
totalCount = 0
file_xml= ''
if os.path.isfile("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    file_xml = "/home/fabbro/Documents/Cartella/fileWiki.xml"
elif os.path.isfile("/home/fabbro/Documents/Big-Data/file.xml"):
    file_xml = "/home/fabbro/Documents/Big-Data/file.xml"

for event, elem in etree.iterparse(file_xml):
    text = ""
    title=""
    if('title' in str(elem.tag)):
        #title_list.append(elem.text)
        #titoliTxt.write(elem.text+"\n")
        if(elem.text is not None):
            title = elem.text.lower()
            title = " ".join([word for word in title.split() if word not in stop_words])
            lista.append(title)
            #print(title)

    if('text' in str(elem.tag)):
        #text_list.append(elem.text)
        if(elem.text is not None):
            text = elem.text.lower()
            text = " ".join([word for word in text.split() if word not in stop_words])
            lista.append(text)
            #print(text)
            
    if len(lista) == 2:
        writer.writerow([lista[0], lista[1]])
        lista=[]
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount)+" rows written of 356,900,000", end="\r") 
            pbar.update(1)
    
    """ if totalCount > 1 and (totalCount % 356900000000) == 0:
            print("{:,}".format(totalCount)+"%")
            pbar.update(1) """
    '''if totalCount > 35000:
        break'''
f.close()
pbar.close()
print("-------------------------------------------DONE------------------------------------------------------")


FileNotFoundError: [Errno 2] No such file or directory: ''

----
Removing the sections titles and break if we are at the end of the part of the page we want to parse (some pages have a lot of sections that we don't want to parse)

In [22]:
def removeNestedParentheses(s):
    lines = s.split('\n')
    print(len(lines))
    ret = []
    for line in lines:
        curr_line = ''
        skip = 0
        for i in line:
            if i == '[':
                skip += 1
            elif i == ']'and skip > 0:
                skip -= 1
            elif skip == 0:
                curr_line += i
        ret.append(curr_line+"\n")
    return '\n'.join(ret)

text = '''
I has a template! {{foo|bar|baz|eggs=spam}} See it <nowiki>or no<!-- revealed --></nowiki>[[File:wiki.png|thumb|Wikipedia logo]]? [[fox]] and [[dog|puppy]] New York also has [[public transport|public transportation]].


New York is a state in the Northeastern United States and is the 27th-most extensive, fourth-most populous, and seventh-most densely populated U.S. state. New York is bordered by New Jersey and Pennsylvania to the south and Connecticut, Massachusetts, and Vermont to the east. The state has a maritime border in the Atlantic Ocean with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the west and north. The state of New York, with an estimated 19.8 million residents in 2015, is often referred to as New York State to distinguish it from New York City, the state's most populous city and its economic hub.
With an estimated population of 8.55 million in 2015, New York City is the most populous city in the United States and the premier gateway for legal immigration to the United States. The New York City Metropolitan Area is one of the most populous urban agglomerations in the world. New York City is a global city, exerting a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment, its fast pace defining the term New York minute. The home of the United Nations Headquarters, New York City is an important center for international diplomacy and has been described as the cultural and financial capital of the world, as well as the world's most economically powerful city. New York City makes up over 40% of the population of New York State. Two-thirds of the state's population lives in the New York City Metropolitan Area, and nearly 40% live on Long Island. Both the state and New York City were named for the 17th century Duke of York, future King James II of England. The next four most populous cities in the state are Buffalo, Rochester, Yonkers, and Syracuse, while the state capital is Albany.
The earliest Europeans in New York were French colonists and Jesuit missionaries who arrived southward from settlements at Montreal for trade and proselytizing. New York had been inhabited by tribes of Algonquian and Iroquoian-speaking Native Americans for several hundred years by the time Dutch settlers moved into the region in the early 17th century. In 1609, the region was first claimed by Henry Hudson for the Dutch, who built Fort Nassau in 1614 at the confluence of the Hudson and Mohawk rivers, where the present-day capital of Albany later developed. The Dutch soon also settled New Amsterdam and parts of the Hudson Valley, establishing the colony of New Netherland, a multicultural community from its earliest days and a center of trade and immigration. The British annexed the colony from the Dutch in 1664. The borders of the British colony, the Province of New York, were similar to those of the present-day state.
Many landmarks in New York are well known to both international and domestic visitors, with New York State hosting four of the world's ten most-visited tourist attractions in 2013: Times Square, Central Park, Niagara Falls (shared with Ontario), and Grand Central Terminal. New York is home to the Statue of Liberty, a symbol of the United States and its ideals of freedom, democracy, and opportunity. In the 21st century, New York has emerged as a global node of creativity and entrepreneurship, social tolerance, and environmental sustainability. New York's higher education network comprises approximately 200 colleges and universities, including Columbia University, Cornell University, New York University, and Rockefeller University, which have been ranked among the top 35 in the world.


== History ==


=== 16th century ===
In 1524, Giovanni da Verrazzano, an Italian explorer in the service of the French crown, explored the Atlantic coast of North America between the Carolinas and Newfoundland, including New York Harbor and Narragansett Bay. On April 17, 1524 Verrazanno entered New York Bay, by way of the Strait now called the Narrows into the northern bay which he named Santa Margherita, in honour of the King of France's sister. Verrazzano described it as "a vast coastline with a deep delta in which every kind of ship could pass" and he adds: "that it extends inland for a league and opens up to form a beautiful lake. This vast sheet of water swarmed with native boats". He landed on the tip of Manhattan and perhaps on the furthest point of Long Island. Verrazanno's stay in this place was interrupted by a storm which pushed him north towards Martha's Vineyard.
In 1540 French traders from New France built a chateau on Castle Island, within present-day Albany; due to flooding, it was abandoned the next year. In 1614, the Dutch under the command of Hendrick Corstiaensen, rebuilt the French chateau, which they called Fort Nassau. Fort Nassau was the first Dutch settlement in North America, and was located along the Hudson River, also within present-day Albany. The small fort served as a trading post and warehouse. Located on the Hudson River flood plain, the rudimentary "fort" was washed away by flooding in 1617, and abandoned for good after Fort Orange (New Netherland) was built nearby in 1623.


=== 17th century ===

Henry Hudson's 1609 voyage marked the beginning of European involvement with the area. Sailing for the Dutch East India Company and looking for a passage to Asia, he entered the Upper New York Bay on September 11 of that year. Word of his findings encouraged Dutch merchants to explore the coast in search for profitable fur trading with local Native American tribes.
During the 17th century, Dutch trading posts established for the trade of pelts from the Lenape, Iroquois, and other tribes were founded in the colony of New Netherland. The first of these trading posts were Fort Nassau (1614, near present-day Albany); Fort Orange (1624, on the Hudson River just south of the current city of Albany and created to replace Fort Nassau), developing into settlement Beverwijck (1647), and into what became Albany; Fort Amsterdam (1625, to develop into the town New Amsterdam which is present-day New York City); and Esopus, (1653, now Kingston). The success of the patroonship of Rensselaerswyck (1630), which surrounded Albany and lasted until the mid-19th century, was also a key factor in the early success of the colony. The English captured the colony during the Second Anglo-Dutch War and governed it as the Province of New York. The city of New York was recaptured by the Dutch in 1673 during the Third Anglo-Dutch War (1672–1674) and renamed New Orange. It was returned to the English under the terms of the Treaty of Westminster a year later.


== References ==


== Further reading ==

French, John Homer (1860). Historical and statistical gazetteer of New York State. Syracuse, New York: R. Pearsall Smith. OCLC 224691273. (Full text via Google Books.)
New York State Historical Association (1940). New York: A Guide to the Empire State. New York City: Oxford University Press. ISBN 978-1-60354-031-5. OCLC 504264143. (Full text via Google Books.)


== External links ==
New York at DMOZ
 Geographic data related to New York at OpenStreetMap'''



section_title_re = re.compile("^=+\s+.*\s+=+$")
content = ''
skip = False

for l in text.splitlines():
    line = l.strip()
    if "= references =" in line.lower():
        skip = True  # replace with break if this is the last section
        break
    if "= further reading =" in line.lower():
        skip = True  # replace with break if this is the last section
        continue
    if section_title_re.match(line):
        skip = False
        continue
    if skip:
        continue
    content+=line+'\n'
    
""" for x in content:
    if x != "":
        print(x.lower()+"\n")
 """
 
withoutPipeLinks = (re.sub(r"\[\[[^|\]]*\|([^|\]]*)]]", r"\1", content, flags=re.S))
withOutComments= re.sub('<!--.*?-->', '', withoutPipeLinks, flags=re.S)
cleaned= re.sub('<.*?>', '', withOutComments, flags=re.S)
testoPulito = removeNestedParentheses(cleaned)
wikicode = mwparserfromhell.parse(testoPulito)
templates = wikicode.filter_templates()
text = " ".join([testo for testo in testoPulito.split() if testo not in templates])
print(text)


22
I has a template! See it or no? and puppy New York also has public transportation. New York is a state in the Northeastern United States and is the 27th-most extensive, fourth-most populous, and seventh-most densely populated U.S. state. New York is bordered by New Jersey and Pennsylvania to the south and Connecticut, Massachusetts, and Vermont to the east. The state has a maritime border in the Atlantic Ocean with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the west and north. The state of New York, with an estimated 19.8 million residents in 2015, is often referred to as New York State to distinguish it from New York City, the state's most populous city and its economic hub. With an estimated population of 8.55 million in 2015, New York City is the most populous city in the United States and the premier gateway for legal immigration to the United States. The New York City Metropolitan Area 

----
Idee per andare avanti:
1. Eliminare prima le doppie graffe (di apertura e chiusura) eliminando il loro contenuto

2. Capire come gestire il comportamento delle stringhe tipo `[[political movement|movement]]` dove l'importante è contenuto dopo la pipe.

3. Eliminare qualsiasi segno di punteggiature (creando una lista di tutti i simboli di punteggiatura) andandoli a rimpiazzare con un singolo spazio in modo che ogni parola sia distanziata effettivamente dall'altra

4. Effettuare un replace di tutti gli spazi (ce ne serve solo uno tra una parola e l'altra, non di più), basta usare la classe `re`

5. Ora che abbiamo tutto l'occorrente, salvare su un file csv ed effettuare l'analisi con _pyspark_

----
_Domande al professore:_

__Risposte da scrivere nella documentazione finale__

1. I numeri sono visti come parole? Sì

2. Teniamo conto dei titoli dei sotto-capitoli? No

3. Teniamo conto delle ''references''? No

4. Posso usare PonyORM?

----

In [43]:
def removeNestedParentheses(s):
    lines = s.split('\n')
    ret = []
    for line in lines:
        curr_line = ''
        skip = 0
        for i in line:
            if i == '[' or i == '{' :
                skip += 1
            elif (i == ']' or i == '}' ) and skip > 0:
                skip -= 1
            elif skip == 0:
                curr_line += i
        ret.append(curr_line+"\n")
    return '\n'.join(ret)

def cleanUp(text):
    section_title_re = re.compile("^=+\s+.*\s+=+$")
    content = ''
    skip = False

    for l in text.splitlines():
        line = l.strip()
        if "= references =" in line.lower():
            skip = True  # replace with break if this is the last section
            break
        if "= further reading =" in line.lower():
            skip = True  # replace with break if this is the last section
            continue
        if section_title_re.match(line):
            skip = False
            continue
        if skip:
            continue
        content+=line+'\n'
        
    """ for x in content:
        if x != "":
            print(x.lower()+"\n")
    """
    
    withoutPipeLinks = (re.sub(r"\[\[[^|\]]*\|([^|\]]*)]]", r"\1", content, flags=re.S))
    withOutComments= re.sub('<!--.*?-->', '', withoutPipeLinks, flags=re.S)
    cleaned= re.sub('<.*?>', '', withOutComments, flags=re.S)
    testoPulito = removeNestedParentheses(cleaned)
    wikicode = mwparserfromhell.parse(testoPulito)
    templates = wikicode.filter_templates()
    text = " ".join([testo for testo in testoPulito.split() if testo not in templates])
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower

Pulisco per bene il file tenendo soltanto le parole, pulendole da tutti i tag (parentesi quadrate, ad esempio) e da tutti i simboli di punteggiatura.
Inoltre vado anche a cancellare i paragrafi ed i loro titoli. Da qui si creerá un metodo che verrá utilizzato 

In [40]:
def pulizia(text):
    
    finalText = ''
    clearedText = ''
    for x in text.splitlines():
        withoutGraphParenthesis = re.sub(r'\{\{[^\}]*\}\}', '', text)
        withOutFile = re.sub(r'\[\[file:[^\]]*\]\]', '', withoutGraphParenthesis)
        withoutPipe= re.sub(r"\[\[[^|\]]*\|([^|\]]*)]]", r"\1", withOutFile)
        withoutParenthesis = re.sub(r'\[\[.*?\]\]', '', withoutPipe)
        withOutRef = re.sub(r'<\/?ref[^>]*>', '', withoutParenthesis)
        withOutComments= re.sub('<!--.*?-->', '', withOutRef)
        withOutClosedRef = re.sub(r'</ref>', '', withOutComments)
        clearedText = clearedText+"".join(withOutClosedRef)
        
    content = []
    skip = False
    for l in clearedText.splitlines():
        line = l.strip()
        if "=references=" in line.lower():
            skip = True  # replace with break if this is the last section
            break
        if "= further reading =" in line.lower():
            skip = True  # replace with break if this is the last section
            continue
        """ if section_title_re.match(line):
            skip = False
            continue """
        if skip:
            continue
        content.append(line)

    for x in content:
        if x != "":
            cleared = re.sub(r'==+.*?==+', '', x)
            punctuation_free = re.sub(r'[^\w\s]', '', cleared)
            finalText = finalText + punctuation_free.lower().strip() + " "
    finalText = ' '.join(finalText.split())
    finalText = re.sub(r'[^\w\s]', '', finalText)
    return finalText.lower

Vado a creare un file csv con i titoli della pagina e le parole in essa contenute. Vado inoltre ad eliminare tutte le stop words.

In [44]:
f= open("file.csv", "w+")
writer = csv.writer(f)

fileWords = open("fileWords.csv", "w+")
fileTitles = open("fileTitles.csv", "w+")

wordsWriter = csv.writer(fileWords)
titlesWriter = csv.writer(fileTitles)

writer.writerow(header)
section_title_re = re.compile("^=+\s+.*\s+=+$")
skip = False

i = 0
j = 0
totalCount = 0
titles_list = []
words_list = []
file_xml= ''
if os.path.isfile("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    file_xml = "/home/fabbro/Documents/Cartella/fileWiki.xml"
elif os.path.isfile("/home/fabbro/Documents/Big-Data/file.xml"):
    file_xml = "/home/fabbro/Documents/Big-Data/file.xml"
else:
    print("File non trovato")
tupla_titolo = ()
tupla_testo = ()
for event, elem in etree.iterparse(file_xml):
    text = ""
    title=""
    
    if('title' in str(elem.tag)):
        if(elem.text is not None):
            i = i+1
            title = elem.text.lower()
            #titles_list.append(title)
            #print("Count title: "+str(i)+" "+title.lower())
            titlesWriter.writerow([i, title])

    if('text' in str(elem.tag)):
        if(elem.text is not None):
            j = j+1
            text = elem.text.lower()
            text = " ".join([word for word in text.split() if word not in stop_words])
            #print("Count text: " + str(j) + " "+pulizia(text))
            text = cleanUp(text)
            #words_list.append(text)
            wordsWriter.writerow([j, text])
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            percents = (totalCount/356900000)*100
            print("{:,}".format(totalCount)+f" rows written of 356,900,000, percents: {percents:.2f}%", end="\r")
#print("Are rows and titles equals? "+str(len(words_list) == len(titles_list)))
#print("Words ha la lunghezza di: "+str(len(words_list))+"mentre il contatore: "+str(j))
#print("Titles ha la lunghezza di: "+str(len(titles_list))+"mentre il contatore: "+str(i))

""" if len(lista) == 2:
        writer.writerow([lista[0], lista[1]])
        lista=[] """
        
''' listaTitoli = []
    listaTitoli.append(title)
    print(listaTitoli[0])
    if i == j and text != "":
        print("Counter: "+str(i)+" - title: "+listaTitoli[0]+" text: "+pulizia(text))'''
f.close()
print("-------------------------------------------DONE------------------------------------------------------")

KeyboardInterrupt: 

In [ ]:
f= open("file.csv", "w+")
writer = csv.writer(f)

fileWords = open("fileWords.csv", "w+")
fileTitles = open("fileTitles.csv", "w+")

wordsWriter = csv.writer(fileWords)
titlesWriter = csv.writer(fileTitles)

writer.writerow(header)
section_title_re = re.compile("^=+\s+.*\s+=+$")
skip = False

i = 0
j = 0
totalCount = 0
titles_list = []
words_list = []
file_xml= ''
if os.path.isfile("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    file_xml = "/home/fabbro/Documents/Cartella/fileWiki.xml"
elif os.path.isfile("/home/fabbro/Documents/Big-Data/file.xml"):
    file_xml = "/home/fabbro/Documents/Big-Data/file.xml"
else:
    print("File non trovato")
tupla_titolo = ()
tupla_testo = ()
for event, elem in etree.iterparse(file_xml):
    text = ""
    title=""
    
    if('title' in str(elem.tag)):
        if(elem.text is not None):
            i = i+1
            title = elem.text.lower()
            titlesWriter.writerow([i, title])

    if('text' in str(elem.tag)):
        if(elem.text is not None):
            j = j+1
            text = elem.text.lower()
            text = " ".join([word for word in text.split() if word not in stop_words])
            text = cleanUp(text)
            wordsWriter.writerow([j, text])
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            percents = (totalCount/356900000)*100
            print("{:,}".format(totalCount)+f" rows written of 356,900,000, percents: {percents:.2f}%", end="\r")
f.close()
print("-------------------------------------------DONE------------------------------------------------------")

In [ ]:
""" 
input_filename = '/home/fabbro/Documents/Big-Data/file.csv'
#input_filename = "/Users/dcarra/Desktop/OneDrive - Università degli Studi di Verona/courses/BigData/PhD_course/2021/lab_notebooks/datasets/short_stories_I.txt"
input_file = sc.textFile(input_filename)

#num_lines = input_file.count()
#print("The number of lines in the input file is:", num_lines)

print(input_file.take(1))
wordsList = input_file.filter(bool) \
            .flatMap(lambda line: line.split(" "))\
            .map(lambda w: w.lower())
wordsKeyValue = wordsList.flatMap(lambda w: (w,1))
print(wordsKeyValue.take(5)) """
inputFile = open("/home/fabbro/Documents/Big-Data/file.csv", "r")
inputFile =inputFile.read().splitlines()
for x in range(5):
    print(inputFile[x])
    print("-------------------------------")

Title,Text
-------------------------------


IndexError: list index out of range

In [7]:
num_Words = len(linecache.getlines("fileWords.csv")) 
print(num_Words)
numTitles = len(linecache.getlines("fileTitles.csv"))
print(numTitles)

NameError: name 'linecache' is not defined

In [1]:
# Specifica il nome del file CSV e la dimensione del blocco
from time import sleep


file_path = "fileWords.csv"
chunk_size = 10

# Utilizza un ciclo per leggere il file CSV in blocchi
num_rows = 0
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    num_rows += len(chunk)
    print(str(chunk.iloc[:,1])+'\r')
    sleep(5)
print("Il file CSV contiene {:,}".format(num_rows), "righe.", end="\r")
# Stampa il numero di righe nel file CSV



NameError: name 'pd' is not defined

In [15]:
# Specifica il nome del file CSV e la dimensione del blocco
file_path = "fileTitles.csv"
chunk = 100000

# Utilizza un ciclo per leggere il file CSV in blocchi
num_righe = 0
for chunks in pd.read_csv(file_path, chunksize=chunk):
    num_righe += len(chunks)
    print(str(chunks.iloc[:,1])+'\n')
    if num_righe > 100:
        break
print("Il file CSV contiene {:,}".format(num_righe), "righe.", end="\r")
# Stampa il numero di righe nel file CSV

0                             anarchism
1                    afghanistanhistory
2                  afghanistangeography
3                     afghanistanpeople
4             afghanistancommunications
                      ...              
99995             rockleigh, new jersey
99996            rutherford, new jersey
99997             teterboro, new jersey
99998    upper saddle river, new jersey
99999            wood-ridge, new jersey
Name: accessiblecomputing, Length: 100000, dtype: object



### Inizio ad utilizzare PySpark utilizzando SparkSession

In [3]:
spark = SparkSession.builder.appName("mySparkApp").getOrCreate()

23/03/29 16:37:58 WARN Utils: Your hostname, kerah resolves to a loopback address: 127.0.1.1; using 192.168.178.79 instead (on interface wlp38s0)
23/03/29 16:37:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 16:37:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<b> Carico i due file come DF </b> <i> poi unendoli in un unico DF tenendo come chiave d'unione l'index</i>. Vado quindi a pulire quel che é stato generato (una cartella) mantenendo solo quel che ci serve a noi: il file csv con i dati.

In [4]:
titles_df = spark.read.csv("fileUno.csv", header=True, inferSchema=True)
text_df = spark.read.csv("fileDue.csv", header=True, inferSchema=True)

merged_df = titles_df.join(text_df, "index")

merged_df.write.csv("merged_file", header=True)

for filename in os.listdir("merged_file"):
    if filename.endswith(".csv"):
        os.rename(os.path.join("merged_file", filename), os.path.join("", "merged.csv"))
        shutil.rmtree("merged_file")

Ora bisogna mappare le parole (contando quante ce ne sono) presenti in ogni cella del file CSV. Stampo inoltre anche il titolo per far capire dove siamo arrivati.

In [5]:
df = spark.read.format("csv").option("header", "true").load("merged.csv")
with open("countedWords.csv", "w+") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "Word", "Counter"])
    for x in range(df.count()):
        title = df.collect()[x]["Title"]
        word = []
        counter = []
        #print("----------- "+df.collect()[x]["Title"]+" -----------")
        cella = df.collect()[x]["Text"]
        words_df = spark.createDataFrame([(cella,)], ['text'])
        words = words_df.select(explode(split(lower('text'), ' ')).alias('word')) \
                        .groupBy('word').agg(count('*').alias('count')) \
                        .orderBy('count', ascending=False)
        word_counts = words.rdd.map(lambda row: (row['word'], row['count'])).collectAsMap()
        for x in word_counts:
            #print("Parola: {}, valore: {}".format(x, word_counts[x]))
            word.append(x)
            counter.append(word_counts[x])
        writer.writerow([title, word, counter])

<b>L’output deve contenere, per ogni parola, la lista di pagine di wikipedia
che contengono quella parola</b>

In [6]:
df = spark.read.format("csv").option("header", "true").load("merged.csv")

with open("reversedIndexing.csv", "w+") as f:
    writer = csv.writer(f)
    writer.writerow(["Word", "Titles"])
    for x in range(df.count()):
        title = df.collect()[x]["Title"]
        word = []
        counter = []
        #print("----------- "+df.collect()[x]["Title"]+" -----------")
        cella = df.collect()[x]["Text"]
        words_df = spark.createDataFrame([(cella,)], ['text'])
        words = words_df.select(explode(split(lower('text'), ' ')).alias('word')) \
                        .groupBy('word').agg(count('*').alias('count')) \
                        .orderBy('count', ascending=True)
        word_counts = words.rdd.map(lambda row: (row['word'], row['count'])).collectAsMap()
        for x in word_counts:
            #print("Parola: {}, valore: {}".format(x, word_counts[x]))
            word.append(x)
            counter.append(word_counts[x])
        writer.writerow([title, word, counter])

Leggiamo tutte le parole presenti nella colonna "Text" del file CSV. Utilizziamo un set in quanto non vogliamo che ci siano parole ripetute. Poi lo trasformiamo in una lista per renderlo indicizzabile.

In [7]:
df = spark.read.format("csv").option("header", "true").load("merged.csv")
word_list = []
tempSet = set()
for x in range(df.count()):
    tempList = []
    if len(df.collect()[x]["Text"].split()) > 1:
        tempList= df.collect()[x]["Text"].lower().split()
        for x in tempList:
            tempSet.add(x)
    else:
        tempSet.add(df.collect()[x]["Text"].lower())
word_list = list(tempSet)

Ora controllo quali parole sono presenti nel file finale da controllare. Se le parole sono presenti, aggiungo il titolo ad una lista. Si creerá infine una riga in un file CSV con la parola e la lista di titoli.

In [8]:
df = spark.read.format("csv").option("header", "true").load("merged.csv")
with open("reversedIndexing.csv", "w+") as f:
    writer = csv.writer(f)
    writer.writerow(["Word", "Titles"])
    #controllo parola per parola
    for x in word_list:
        title_list = []
        #controllo ogni singola riga del file CSV
        for y in range(df.count()):
            if x in df.collect()[y]["Text"].lower().split():
                #print("La parola '{}' è presente nel titolo '{}'".format(x, df.collect()[y]["Title"]))
                title_list.append(df.collect()[y]["Title"])
        writer.writerow([x, title_list])

23/03/29 16:38:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
